# IMPORT LIBRARY & DATA

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/DDB Telkom/Customer Research Pijar Mahir/Pijar Mahir Experience Questionnaire (Responses).xlsx')

# EXPLORATION AND CLEANING

In [ ]:
df.head()

,Timestamp,Email address,Nama,Nomor HP,Pas Foto / Foto dengan wajah anda yang terlihat jelas,Jenis Kelamin,Tanggal Lahir,Kota Tempat anda Tinggal,Provinsi Tempat anda Tinggal,Pendapatan anda selama 1 Bulan,Pendidikan Terakhir,Pekerjaan,Darimana anda mengetahui Pijar Mahir?,Apakah anda sudah pernah membeli pelatihan di Pijar Mahir?,Apa saja media sosial Pijar Mahir yang sudah anda ikuti?,Berapa kali anda pernah membeli pelatihan di Pijar Mahir?,Mengapa anda ingin/sudah mengikuti pelatihan di Pijar Mahir?,Bagaimana harga pelatihan di Pijar Mahir?,Bagaimana kelengkapan pelatihan di Pijar Mahir?,Bagaimana materi pelatihan di Pijar Mahir?,Bagaimana user journey di Pijar Mahir?,Bagaimana tampilan website Pijar Mahir?,Bagaimana kecepatan loading website Pijar Mahir?,Apa yang anda sukai dari Pijar Mahir?,Apakah anda berencana untuk mengikuti pelatihan di Pijar Mahir lagi?,Kategori apa yang anda sukai?,Bagaimana perasaan anda ketika website loading terlalu lama?,Bagaimana perasaan anda ketika berhasil menyelesaikan pelatihan?,Bagaimana perasaan anda ketika sedang membuka website Pijar Mahir?,Bagaimana ekspetasi anda tentang Pijar Mahir secara keseluruhan?,Seberapa ingin anda menambah skill dan wawasan baru?,Seberapa ingin anda memperluas skill dan wawasan anda?,Seberapa puas anda dengan pelayanan customer service Pijar Mahir?,Seberapa puas anda dengan konten sosial media Pijar Mahir?,Seberapa puas anda dengan event webinar dari Pijar Mahir?,Seberapa puas anda dengan promo pelatihan dari Pijar Mahir?,Apakah anda merasa kesulitan mencari waktu luang untuk memulai/melanjutkan pelatihan?,Kapan biasanya anda memulai/melanjutkan pelatihan yang sedang anda ambil?,Silahkan ceritakan testimoni anda tentang Sebelum dan Sesudah mengenal Pijar Mahir,Jika anda memiliki saran silahkan untuk menceritakan saran anda pada kolom dibawah ini *kosongkan jika tidak ada,Jika anda memiliki kritik silahkan untuk menceritakan kritik anda pada kolom dibawah ini *kosongkan jika tidak ada,Seberapa besar keinginan anda merekomendasikan Pijar Mahir?
0,2021-08-16 17:00:26.834,aminmaskur45@gmail.com,Amin Maskur,081548469666,https://drive.google.com/open?id=180VbhTcYpdFi...,Laki-Laki,2001-10-25 00:00:00,Cilacap,Jawa Tengah,< Rp. 1.500.000,SMA,Belum/Tidak Bekerja,Website Prakerja,Ya,"Instagram, Facebook, Twitter, Linkedin",Lebih dari 1 kali,"Menambah Skill dan Wawasan baru, Menyelesaikan...",5,5,5,4,4,5,"Media Sosial, Tampilan Website, Kecepatan Load...",Mungkin,"Kartu Prakerja, Mahir Teknologi, Mahir Digital",Biasa saja,Ingin mengambil pelatihan lagi,Nyaman,Terpenuhi,4,5,5,5,5,5,Tidak,Malam (18.00 - 24.00),Pelatihan Cukup baik dan dapat diterima dengan...,NaN,NaN,NaN
1,2021-08-16 17:00:33.211,ricky.almahenrah@gmail.com,Ricky Almahenrah,08118737999,https://drive.google.com/open?id=1OzHsULPeu8Na...,Laki-Laki,1999-04-09 00:00:00,Bekasi,Jawa Barat,> Rp. 5.000.000,SMA,Pegawai Swasta,Teman / Kerabat,Ya,Instagram,1 kali,Menambah Skill dan Wawasan baru,4,4,5,4,4,3,"Media Sosial, Tampilan Website, Harga, Materi ...",Mungkin,"Mahir Telekomunikasi, Mahir Digital, Mahir Mar...",Kesal,Senang,Nyaman,Terpenuhi,5,5,4,5,4,5,Ya,Malam (18.00 - 24.00),Sejauh ini bagus dan oke oke aja/,NaN,NaN,NaN
2,2021-08-16 17:00:44.502,wismaya22@gmail.com,yunita sari,083874355222,NaN,Perempuan,1983-06-17 00:00:00,depok,jawa barat,Rp. 1.500.000 - Rp. 3.000.000,Sarjana,Pegawai Swasta,Website Prakerja,Ya,tidak mengikuti media sosial Pijar Mahir,1 kali,Menyelesaikan Pelatihan untuk Insentif di Kart...,3,4,4,4,4,4,"Tampilan Website, Kelengkapan Pelatihan, Harga...",Ya,"Kartu Prakerja, Mahir Telekomunikasi, Mahir Te...",Kesal,Ingin mengambil pelatihan lagi,Nyaman,Terpenuhi,5,5,4,4,4,4,Tidak,Malam (18.00 - 24.00),pelatihannya sangat menambah wawasab,NaN,NaN,NaN
3,2021-08-16 17:00:54.541,agustinnisa148@gmail.com,Nisa agustin,085281515066,https://drive.google.com/open?id=1XykooZ-W7xL6...,Perempuan,2001-08-23 00:00:00,Bandung,Jawa barat,< Rp. 1.500.000,SMA,Lainnya,Website Prakerja,Ya,"Instagram, Facebook",1 kal

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9947 entries, 0 to 9946
Data columns (total 42 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype         
---  ------                                                                                                                                         --------------  -----         
 0   Timestamp                                                                                                                                      9947 non-null   datetime64[ns]
 1   Email address                                                                                                                                  9947 non-null   object        
 2   Nama                                                                                                                                           9947 non-null   object       

In [ ]:
df.columns = ['timestamp', 'email', 'nama', 'telepon', 'pasFoto', 'jenisKelamin', 'tanggalLahir', 'kota',
              'provinsi', 'pendapatan', 'pendidikanTerakhir', 'pekerjaan', 'mengetahuiPijarMahir', 'membeliPelatihan', 'medsosPijarMahir', 'jumlahPembelian',
              'alasanMengikutiPelatihan', 'hargaPelatihan', 'variasiPelatihan', 'materiPelatihan', 'userJourney', 'tampilanWebsite', 'kecepatanWebsite', 'disukaiDariPijarMahir',
              'berencanaMengikutiPelatihan', 'kategoriDisukai', 'perasaanLoadingLama', 'perasaanMenyelesaikanPelatihan', 'perasaanMembukaWebPijarMahir', 'ekspetasiPijarMahir', 'inginMenambahSkill', 'inginMemperdalamSkill',
              'kepuasanCustomerService', 'kepuasanKontenSosialMedia', 'kepuasanEventWebinar', 'kepuasanPromoPelatihan', 'kesulitanWaktuLuang', 'jamBelajar', 'testimoni', 'saran',
              'kritik', 'merekomendasiPijarMahir']

In [ ]:
df[['saran', 'kritik']] = df[['saran', 'kritik']].fillna('tidak ada')

# PREPROCESSING KRITIK & SARAN

INSTALL LIBRARY

In [ ]:
!pip install nltk
!pip install Sastrawi
!pip install regex
!pip install swifter
!pip install tweet-preprocessor

IMPORT LIBRARY

In [ ]:
#nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords

#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import swifter

#regex
import string
import re

#tweet preprocessor
import preprocessor as p

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

ASSIGN DATA

In [ ]:
df_ks = pd.DataFrame(df[['kritik', 'saran']])

CASE FOLDING AND CHANGE DTYPES TO STRING

In [ ]:
df_ks['kritik'] = df_ks['kritik'].str.lower()
df_ks['saran'] = df_ks['saran'].str.lower()

In [ ]:
df_ks = df_ks.astype(str)
df = df.astype(str)

NORMALIZATION

remove characters other than letters

In [ ]:
def normalization(text):
    p.set_options(p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER, p.OPT.SMILEY, p.OPT.URL)
    res = p.clean(text)
    return res

In [ ]:
#using tweet_preprocessor library
df_ks['kritik'] = df_ks['kritik'].apply(normalization)
df_ks['saran'] = df_ks['saran'].apply(normalization)

In [ ]:
#using regex

#remove special char
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

# remove single char
def remove_single_char(text):
    return re.sub(r"(\b[a-zA-Z]\b)|([-.])", "", text)

df_ks['kritik'] = df_ks['kritik'].apply(remove_tweet_special)
df_ks['saran'] = df_ks['saran'].apply(remove_tweet_special)
df_ks['kritik'] = df_ks['kritik'].apply(remove_number)
df_ks['saran'] = df_ks['saran'].apply(remove_number)
df_ks['kritik'] = df_ks['kritik'].apply(remove_punctuation)
df_ks['saran'] = df_ks['saran'].apply(remove_punctuation)
df_ks['kritik'] = df_ks['kritik'].apply(remove_whitespace_multiple)
df_ks['saran'] = df_ks['saran'].apply(remove_whitespace_multiple)
df_ks['kritik'] = df_ks['kritik'].apply(remove_single_char)
df_ks['saran'] = df_ks['saran'].apply(remove_single_char)

TOKENIZING (NLTK)

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df_ks['kritik'] = df_ks['kritik'].apply(word_tokenize_wrapper)
df_ks['saran'] = df_ks['saran'].apply(word_tokenize_wrapper)

SLANG WORD NORMALIZATION

In [ ]:
kamus_alay = pd.read_csv('/content/new_kamusalay.csv', encoding='latin-1')

kamus_alay_dict = {}

for index, row in kamus_alay.iterrows():
  if row[0] not in kamus_alay:
    kamus_alay_dict[row[0]] = row[1]

def normalized_alay(document):
  return [kamus_alay_dict[term] if term in kamus_alay_dict else term for term in document]

df_ks['kritik'] = df_ks['kritik'].apply(normalized_alay)
df_ks['saran'] = df_ks['saran'].apply(normalized_alay)

FILTERING

In [ ]:
#ambil stopword bahasa indonesia
list_stopwords = stopwords.words('indonesian')

#menambahkan stopword secara manual
list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'no', 'je', 'om', 'pru', 'sch',
                       'injirrr', 'ah', 'oena', 'bu', 'eh'])

list_stopwords = set(list_stopwords)

def stopwords_removal(document):
  return [word for word in document if word not in list_stopwords]

df_ks['kritik'] = df_ks['kritik'].apply(stopwords_removal)
df_ks['saran'] = df_ks['saran'].apply(stopwords_removal)

STEMMING

cari kata dasar dari sebuah kata

stem kritik

In [ ]:
#buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stemmed wrapper
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in df_ks['kritik']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

# print(len(term_dict))
# print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
#     print(term,":" ,term_dict[term])
    
# print(term_dict)
# print("------------------------")

#memmulai stemming
def apply_stemmed_term(document):
  return [term_dict[term] for term in document]

df_ks['kritik'] = df_ks['kritik'].swifter.apply(apply_stemmed_term)

Pandas Apply:   0%|          | 0/9947 [00:00<?, ?it/s]

stem saran

In [ ]:
#buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stemmed wrapper
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in df_ks['saran']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

# print(len(term_dict))
# print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
#     print(term,":" ,term_dict[term])
    
# print(term_dict)
# print("------------------------")

#memmulai stemming
def apply_stemmed_term(document):
  return [term_dict[term] for term in document]

df_ks['saran'] = df_ks['saran'].swifter.apply(apply_stemmed_term)

Pandas Apply:   0%|          | 0/9947 [00:00<?, ?it/s]

# UNSUPERVISED CLASSIFICATION KRITIK & SARAN USING WORD EMBEDDING

INSTALL & IMPORT LIBRARY

In [ ]:
!pip install spacy

In [ ]:
from sklearn import model_selection, metrics, neighbors
from scipy import spatial
import spacy
import numpy as np

LOAD PRETRAINED FAST-TEXT MODEL

In [ ]:
!python -m spacy init-model de id --vectors-loc /content/drive/MyDrive/FastText/cc.id.300.vec

✔ Successfully created model
⠹ Reading vectors from /content/drive/MyDrive/FastText/cc.id.300.vectcmalloc: large alloc 2400002048 bytes == 0x56309d030000 @  0x7f0bcb561001 0x7f0bc90df54f 0x7f0bc912fb58 0x7f0bc9133b17 0x7f0bc91d2203 0x56309a7a50a4 0x56309a7a4da0 0x56309a819868 0x56309a7a665a 0x56309a814b0e 0x56309a813c35 0x56309a7a673a 0x56309a814b0e 0x56309a813c35 0x56309a6e5eb1 0x56309a816318 0x56309a814235 0x56309a7a673a 0x56309a814d67 0x56309a813c35 0x56309a7a673a 0x56309a818f40 0x56309a813c35 0x56309a813933 0x56309a811da0 0x56309a7a4ea9 0x56309a7a4da0 0x56309a818bb3 0x56309a813c35 0x56309a7a673a 0x56309a814b0e
2000000it [02:35, 12837.58it/s]
✔ Loaded vectors from /content/drive/MyDrive/FastText/cc.id.300.vec
✔ Sucessfully compiled vocab
2000255 entries, 2000000 vectors


In [ ]:
nlp = spacy.load('id')

WORD EMBEDDING

In [ ]:
def embed(tokens, nlp):
  """
  ngembaliin centroid dari token yang di embedding
  
  token yang berada diluar kosa kata akan dikesampingkan.
  array 0 akan dikembalikan jika token tidak valid
  """

  lexemes = (nlp.vocab[token] for token in tokens)

  vectors = np.asarray([
                        lexeme.vector
                        for lexeme in lexemes
                        if lexeme.has_vector
                        and not lexeme.is_stop
                        and len(lexeme.text) > 1
  ])

  if len(vectors) > 0:
    centroid = vectors.mean(axis=0)
  else:
    width = nlp.meta['vectors']['width']
    centroid = np.zeros(width)

  return centroid

PREDICT

In [ ]:
def predict(term, nlp, neigh):
  """
  menghitung jarak dari centroid ke setiap vektor label dan mengembalikan jarak terdekat
  """
  centroid = embed(term, nlp)
  closest_label = neigh.kneighbors([centroid], return_distance=False)[0][0]
  
  return closest_label

In [ ]:
label_names = ['pemasaran', 'situs', 'pelayanan', 'pelatihan']

label_vectors = np.asarray([
                            embed(name.split(' '), nlp)
                            for name in label_names
])

In [ ]:
neigh = neighbors.NearestNeighbors(
    n_neighbors=3,
    metric=spatial.distance.cosine
)
neigh.fit(label_vectors)

preds_kritik = [label_names[predict(term, nlp, neigh)] for term in df_ks['kritik']]

preds_saran = [label_names[predict(term, nlp, neigh)] for term in df_ks['saran']]


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in

In [ ]:
preds_saran

['media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'pelayanan',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'pelayanan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'situs',
 'pelatihan',
 'media sosial',
 'pelatihan',
 'media sosial',
 'pelayanan',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'medi

In [ ]:
preds_kritik

['media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelayanan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'pelatihan',
 'media sosial',
 'media sosial',
 'media sosial',
 'media sosial',
 'm

# FINAL FILTERING

In [ ]:
df['kategoriSaran'] = preds_saran
df['kategoriKritik'] = preds_kritik
df['checkKritik'] = df_ks['kritik']
df['checkSaran'] = df_ks['saran']

CLEANING KRITIK DAN SARAN KOSONG

In [ ]:
df['kritik'] = df['kritik'].apply(remove_single_char)
df['saran'] = df['saran'].apply(remove_single_char)

In [ ]:
# mencari string kosong
df['kritik'][df['kritik'] == ''] = 'tidak ada'
df['saran'][df['saran'] == ''] = 'tidak ada'

# mencari kata takada
df['kritik'][df['kritik'] == 'Takada'] = 'tidak ada'
df['saran'][df['saran'] == 'Takada'] = 'tidak ada'

# mencari kata belum ada ditengah string
df['kritik'][df['kritik'].str.find('belum ada') != -1] = 'tidak ada'
df['saran'][df['saran'].str.find('belum ada') != -1] = 'tidak ada'

df['kritik'][df['kritik'].str.find('Belum ada') != -1] = 'tidak ada'
df['saran'][df['saran'].str.find('Belum ada') != -1] = 'tidak ada'

# mencari kata tidak ada ditengah string
df['kritik'][df['kritik'].str.find('tidak ada') != -1] = 'tidak ada'
df['saran'][df['saran'].str.find('tidak ada') != -1] = 'tidak ada'

df['kritik'][df['kritik'].str.find('Tidak ada') != -1] = 'tidak ada'
df['saran'][df['saran'].str.find('Tidak ada') != -1] = 'tidak ada'

#mengubah tidak ada menjadi -
df['kritik'][df['kritik'] == 'tidak ada'] = '-'
df['saran'][df['saran'] == 'tidak ada'] = '-'

In [ ]:
df['saran'].unique()

array(['-',
       'Saran nyaa mohon di percepat dalam menanggapi setiap kendala di pijar mahir',
       'Lebih banyak promo pelatihan', ...,
       'Ayo kawan, belilah paket pelatihan di Pijar Mahir, banyak pilihan, harga murah dan kursusnya mudah dimengerti ',
       'Untuk saat ini Uda cukukup memuaskan', 'Lanjutkan!!!'],
      dtype=object)

In [ ]:
df[['saran', 'kritik']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9947 entries, 0 to 9946
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   saran   9947 non-null   object
 1   kritik  9947 non-null   object
dtypes: object(2)
memory usage: 155.5+ KB


In [ ]:
df.iloc[26]

timestamp                                                   2021-08-16 17:06:11.431
email                                                      saidsidorejo19@gmail.com
nama                                                                           Said
telepon                                                            3309191910950001
pasFoto                           https://drive.google.com/open?id=1Iw3G6jHJkR8B...
jenisKelamin                                                              Laki-Laki
tanggalLahir                                                    1995-10-19 00:00:00
kota                                                                       Boyolali
provinsi                                                                Jawa Tengah
pendapatan                                                          < Rp. 1.500.000
pendidikanTerakhir                                                              SMA
pekerjaan                                                         Pelajar/Ma

In [ ]:
df['kategoriKritik'][df['kritik'] == '-'] = '-'
df['kategoriSaran'][df['saran'] == '-'] = '-'

In [ ]:
df.drop(columns=['checkKritik', 'checkSaran'], inplace=True)

In [ ]:
df.head()

,timestamp,email,nama,telepon,pasFoto,jenisKelamin,tanggalLahir,kota,provinsi,pendapatan,pendidikanTerakhir,pekerjaan,mengetahuiPijarMahir,membeliPelatihan,medsosPijarMahir,jumlahPembelian,alasanMengikutiPelatihan,hargaPelatihan,variasiPelatihan,materiPelatihan,userJourney,tampilanWebsite,kecepatanWebsite,disukaiDariPijarMahir,berencanaMengikutiPelatihan,kategoriDisukai,perasaanLoadingLama,perasaanMenyelesaikanPelatihan,perasaanMembukaWebPijarMahir,ekspetasiPijarMahir,inginMenambahSkill,inginMemperdalamSkill,kepuasanCustomerService,kepuasanKontenSosialMedia,kepuasanEventWebinar,kepuasanPromoPelatihan,kesulitanWaktuLuang,jamBelajar,testimoni,saran,kritik,merekomendasiPijarMahir,kategoriSaran,kategoriKritik
0,2021-08-16 17:00:26.834,aminmaskur45@gmail.com,Amin Maskur,081548469666,https://drive.google.com/open?id=180VbhTcYpdFi...,Laki-Laki,2001-10-25 00:00:00,Cilacap,Jawa Tengah,< Rp. 1.500.000,SMA,Belum/Tidak Bekerja,Website Prakerja,Ya,"Instagram, Facebook, Twitter, Linkedin",Lebih dari 1 kali,"Menambah Skill dan Wawasan baru, Menyelesaikan...",5,5,5,4,4,5,"Media Sosial, Tampilan Website, Kecepatan Load...",Mungkin,"Kartu Prakerja, Mahir Teknologi, Mahir Digital",Biasa saja,Ingin mengambil pelatihan lagi,Nyaman,Terpenuhi,4,5,5,5,5,5,Tidak,Malam (18.00 - 24.00),Pelatihan Cukup baik dan dapat diterima dengan...,-,-,nan,-,-
1,2021-08-16 17:00:33.211,ricky.almahenrah@gmail.com,Ricky Almahenrah,08118737999,https://drive.google.com/open?id=1OzHsULPeu8Na...,Laki-Laki,1999-04-09 00:00:00,Bekasi,Jawa Barat,> Rp. 5.000.000,SMA,Pegawai Swasta,Teman / Kerabat,Ya,Instagram,1 kali,Menambah Skill dan Wawasan baru,4,4,5,4,4,3,"Media Sosial, Tampilan Website, Harga, Materi ...",Mungkin,"Mahir Telekomunikasi, Mahir Digital, Mahir Mar...",Kesal,Senang,Nyaman,Terpenuhi,5,5,4,5,4,5,Ya,Malam (18.00 - 24.00),Sejauh ini bagus dan oke oke aja/,-,-,nan,-,-
2,2021-08-16 17:00:44.502,wismaya22@gmail.com,yunita sari,083874355222,nan,Perempuan,1983-06-17 00:00:00,depok,jawa barat,Rp. 1.500.000 - Rp. 3.000.000,Sarjana,Pegawai Swasta,Website Prakerja,Ya,tidak mengikuti media sosial Pijar Mahir,1 kali,Menyelesaikan Pelatihan untuk Insentif di Kart...,3,4,4,4,4,4,"Tampilan Website, Kelengkapan Pelatihan, Harga...",Ya,"Kartu Prakerja, Mahir Telekomunikasi, Mahir Te...",Kesal,Ingin mengambil pelatihan lagi,Nyaman,Terpenuhi,5,5,4,4,4,4,Tidak,Malam (18.00 - 24.00),pelatihannya sangat menambah wawasab,-,-,nan,-,-
3,2021-08-16 17:00:54.541,agustinnisa148@gmail.com,Nisa agustin,085281515066,https://drive.google.com/open?id=1XykooZ-W7xL6...,Perempuan,2001-08-23 00:00:00,Bandung,Jawa barat,< Rp. 1.500.000,SMA,Lainnya,Website Prakerja,Ya,"Instagram, Facebook",1 kali,"Menambah Skill dan Wawasan baru, Menyelesaikan...",4,4,5,4,5,5,"Harga, Materi yang Menarik",Ya,UMKM,Bingung,Senang,Nyaman,Terpenuhi,5,5,4,4,4,4,Ya,Siang (10.00 - 14.00),"Wah bagus banget ka,, mudah di mengerti dan di...",Saran nyaa mohon di percepat dalam menanggapi ...,-,nan,pelatihan,-
4,2021-08-16 17:01:28.020,ds7909804@gmail.com,Dani setiawan,085227504911,nan,Laki-Laki,1987-04-08 00:00:00,Banjarnegara,Jawa tengah,< Rp. 1.500.000,SMA,Belum/Tidak Bekerja,Website Prakerja,Ya,tidak mengikuti media sosial Pijar Mahir,1 kali,Menambah Skill dan Wawasan baru,5,5,5,5,5,5,Materi yang Menarik,Ya,Mahir Marketing,Biasa saja,Senang,Nyaman,Melebihi Ekspetasi,5,5,5,5,5,5,Ya,Malam (18.00 - 24.00),Sangat senang,-,-,nan,-,-


In [ ]:
9947 - len(df[df['kritik'].str.contains('-')])

1335

In [ ]:
9947 - len(df[df['saran'].str.contains('-')])

3363

EXPORT TO EXCEL

In [ ]:
!pip install XlsxWriter

In [ ]:
writer = pd.ExcelWriter('Customer Research Pijar Mahir.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
df['medsosPijarMahir'].str.get_dummies(sep=', ').rename(lambda x: x, axis='columns')

,Facebook,Instagram,Linkedin,Twitter,tidak mengikuti media sosial Pijar Mahir
0,1,1,1,1,0
1,0,1,0,0,0
2,0,0,0,0,1
3,1,1,0,0,0
4,0,0,0,0,1
...,...,...,...,...,...
9942,0,0,0,0,1
9943,0,0,0,0,1
9944,1,1,0,1,0
9945,1,1,0,0,0
